In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv('train_values.csv'),
                 pd.read_csv('train_labels.csv'))
test = pd.read_csv('test_values.csv')
sample_submission = pd.read_csv('submission_format.csv')

train.shape, test.shape


((260601, 40), (86868, 39))

In [2]:
#train, validation = train_test_split(train, train_size=0.80, test_size=0.20, stratify=train['damage_grade'], random_state=1337)

#train.shape, validation.shape, test.shape



In [3]:
train['damage_grade'].value_counts(normalize=True)

2    0.568912
3    0.334680
1    0.096408
Name: damage_grade, dtype: float64

In [4]:
def wrangle(X):
    X = X.copy()

    X = X.drop(['building_id'], axis=1)

    return X

In [5]:
train = wrangle(train)
#validation = wrangle(validation)
test = wrangle(test)

In [6]:
from sklearn.metrics import accuracy_score

majority_class = train['damage_grade'].mode()[0]
y_pred = [majority_class] * len(train['damage_grade'])

print("Train baseline accuracy is: ", accuracy_score(train['damage_grade'], y_pred))

Train baseline accuracy is:  0.5689118614280068


In [7]:
# majority_class = validation['damage_grade'].mode()[0]
# y_pred = [majority_class] * len(validation['damage_grade'])
#
# print("Validation baseline accuracy is: ", accuracy_score(validation['damage_grade'], y_pred))

In [8]:
target = 'damage_grade'

train_features = train.drop([target], axis=1)
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

categorical_features = train_features.select_dtypes(exclude='number').nunique().index.tolist()


features = numeric_features + categorical_features

In [9]:
y_train = train[target]
X_train = train[features]
# X_validation = validation[features]
# y_validation = validation[target]

X_test = test[features]

In [10]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler, SMOTE
from skopt import BayesSearchCV
from sklearn.model_selection import GridSearchCV

#When hyperparamater tuning, set tune to "True", and mark each model that we want to tune to "True".
tune = False
forest = True
xgboost = False

# forest_distributions = {
#     'model__n_estimators': [50, 100, 150, 200, 250, 300],
#     'model__max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
#     'model__max_features': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
#                             31, 32, 33, 34, 35, 36, 37, 38],
#     'model__min_samples_leaf': [2, 3, 4, 5, 6, 7, 8]
# }
#
# xgboost_distributions = {
#     'model__n_estimators': [50, 60, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300],
#     'model__max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
#     'model__learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05, 0.07, 0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.22, 0.24],
#     'model__min_child_leaf':[1, 2, 3, 4, 5, 6],
#     'model__min_child_weight': [1, 2, 3, 4, 5],
#     'model__colsample_bytree':[0.1, 0.15, 0.2, 0.3, 0.4, 0.50, 0.60, 0.70],
#     'model__subsample':[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
#     'model__gamma':[0, 1, 2, 3, 4, 5, 10, 20],
#     'model__scale_pos_weight': [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70,
#                                 75, 80, 85, 90, 95, 100]
#  }

forest_distributions = {
    'model__n_estimators': [350, 375, 400, 425],
    'model__max_depth': [29, 30, 31, 32, 33],
    'model__max_features': [24, 25, 26, 27],
    'model__min_samples_leaf': [4]
}

xgboost_distributions = {
    'model__n_estimators': [260, 265, 270],
    'model__max_depth': [26, 27],
    'model__learning_rate': [0.08, 0.09],
    'model__min_child_leaf':[3],
    'model__min_child_weight': [4],
    'model__colsample_bytree':[0.2],
    'model__subsample':[0.9, 1],
    'model__gamma':[0],
    'model__scale_pos_weight': [25]
  }

if tune: #If we are hyperparamater tuning, pass no parameters into estimators and find paramaters via GridSearchCV / RandomizedSearchCV

    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', RandomForestClassifier(random_state=1337))])

    xgboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                           ('model', XGBClassifier(seed=1337))])

    # forest_search = BayesSearchCV(
    #     forest_pipeline,
    #     search_spaces=forest_distributions,
    #     n_iter=100,
    #     cv=3,
    #     scoring='f1_micro',
    #     random_state=1337,
    #     n_points=5,
    #     n_jobs=15
    # )
    # xgboost_search = BayesSearchCV(
    #     estimator=xgboost_pipeline,
    #     search_spaces=xgboost_distributions,
    #     n_iter=800,
    #     cv=3,
    #     scoring='f1_micro',
    #     random_state=1337,
    #     n_points=5,
    #     n_jobs=15
    # )

    forest_search = GridSearchCV(
        estimator=forest_pipeline,
        param_grid=forest_distributions,
        cv=3,
        verbose=3,
        scoring='f1_micro',
        n_jobs=15
    )
    xgboost_search = GridSearchCV(
        estimator=xgboost_pipeline,
        param_grid=xgboost_distributions,
        cv=3,
        verbose=3,
        scoring='f1_micro',
        n_jobs=15
    )

    def report_step_forest(optim_result):
        score = forest_search.best_score_
        print("Random Forest best score: %s" % score)

    def report_step_xgboost(optim_result):
        score = xgboost_search.best_score_
        print("XGboost best score: %s" % score)

    #X_train, y_train = RandomOverSampler(sampling_strategy='not majority').fit_resample(X_train, y_train)

    if forest:
        forest_search.fit(X_train, y_train)
        forest_train_pred = forest_search.predict(X_train)
        forest_validation_pred = forest_search.predict(X_validation)
        forest_validation_pred_proba = forest_search.predict_proba(X_validation)

    if xgboost:
        xgboost_search.fit(X_train, y_train)
        xgboost_train_pred = xgboost_search.predict(X_train)
        xgboost_validation_pred = xgboost_search.predict(X_validation)
        xgboost_validation_pred_proba = xgboost_search.predict_proba(X_validation)

    #When hyperparameter tuning, pass our best estimators into votingclassifier. Only run when all 3 models are being tuned.
    if forest and xgboost:
        voting_model = VotingClassifier(estimators=[('forest', forest_search.best_estimator_), #VotingClassifier is Soft Voting/Majority Rule classifier for unfitted estimators
                                                    ('xgboost', xgboost_search.best_estimator_),],
                                        voting='soft') #soft voting per recommendation from sklearn documentation, when used on tuned classifiers.
        voting_model.fit(X_train, y_train)
        # combined_model_pred = voting_model.predict(X_validation)

else: #If we are not hyperparameter tuning, pass in our best params(from previous tuning runs).
    forest_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                            ('model', RandomForestClassifier(random_state=1337,
                                                             n_jobs=15, max_depth=30, max_features=25, min_samples_leaf=4,
                                                             n_estimators=350))])

    xgboost_pipeline = Pipeline([('encoder', ce.OrdinalEncoder()),
                                ('model', XGBClassifier(seed=1337, n_jobs=15, colsample_bytree=0.2, gamma=0,
                                                        learning_rate=0.09, max_depth=26, min_child_leaf=3, min_child_weight=4,
                                                        n_estimators=260, scale_pos_weight=25, subsample=0.9))])


    #X_train, y_train = RandomOverSampler(sampling_strategy='minority').fit_resample(X_train, y_train) #over sample all but the majority class

    forest_pipeline.fit(X_train, y_train)
    forest_train_pred = forest_pipeline.predict(X_train)
    # forest_validation_pred = forest_pipeline.predict(X_validation)
    # forest_validation_pred_proba = forest_pipeline.predict_proba(X_validation)
    forest_test_pred = forest_pipeline.predict(X_test)

    xgboost_pipeline.fit(X_train, y_train)
    xgboost_train_pred = xgboost_pipeline.predict(X_train)
    # xgboost_validation_pred = xgboost_pipeline.predict(X_validation)
    # xgboost_validation_pred_proba = xgboost_pipeline.predict_proba(X_validation)
    xgboost_test_pred = xgboost_pipeline.predict(X_test)

    voting_model = VotingClassifier(estimators=[('forest', forest_pipeline), #VotingClassifier is Soft Voting/Majority Rule classifier for unfitted estimators
                                                ('xgboost', xgboost_pipeline),],
                                    voting='soft') #soft voting per recommendation from sklearn documentation, when used on tuned classifiers.
    voting_model.fit(X_train, y_train)
    # combined_model_pred = voting_model.predict(X_validation)
    combined_model_test_pred = voting_model.predict(X_test)

In [11]:
# from sklearn.metrics import f1_score, classification_report
# if tune:
#
#     if forest:
#         print(forest_search.best_params_, '\n')
#         print("Best Random Forest CV score: ", forest_search.best_score_, '\n')
#         print("Random forest F1-Micro: ", f1_score(y_validation, forest_validation_pred, average='micro'), '\n')
#         print(classification_report(y_validation, forest_validation_pred), '\n')
#
#         print('\n')
#
#     if xgboost:
#         print(xgboost_search.best_params_, '\n')
#         print("Best xgboost CV score: ", xgboost_search.best_score_, '\n')
#         print("Xgboost F1-Micro: ", f1_score(y_validation, xgboost_validation_pred, average='micro'), '\n')
#         print(classification_report(y_validation, xgboost_validation_pred), '\n')
#
#         print('\n')
#
#     if forest and xgboost:
#         print("Voting classifier, final F1-Micro score on validation set: ", f1_score(y_validation, combined_model_pred, average='micro'))
# else:
#     print("Random forest F1-Micro: ", f1_score(y_validation, forest_validation_pred, average='micro'), '\n')
#     print(classification_report(y_validation, forest_validation_pred))
#
#     print('\n')
#
#     print("Xgboost F1-Micro: ", f1_score(y_validation, xgboost_validation_pred, average='micro'), '\n')
#     print(classification_report(y_validation, xgboost_validation_pred))
#
#     print('\n')
#
#     print("Voting classifier, final F1-Micro score on validation set: ", f1_score(y_validation, combined_model_pred, average='micro'))


In [12]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, minimal=True).to_notebook_iframe()
#
# profile

In [14]:
# #combined_model_test_pred = voting_model.predict(X_test)
#xgboost_test_pred = xgboost_search.predict(X_test)
submission = sample_submission.copy()
submission['damage_grade'] = xgboost_test_pred
submission.to_csv('xgboost.csv', index=False)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    xgboost_pipeline.named_steps.model, #prefit estimator
    scoring='f1_micro',
    n_iter=5,
    random_state=1337
)

permuter.fit(forest_pipeline.named_steps.encoder.transform(X_validation), y_validation)
#

In [ ]:
feature_names = X_validation.columns.tolist()
#

In [ ]:
eli5.show_weights(
    permuter,
    top=None,
    feature_names=feature_names
)